# 作業

### 嘗試用 keras 的 DepthwiseConv2D 等 layers 實做 Inverted Residual Block.
   - depthwise's filter shape 爲 (3,3), padding = same
   - 不需要給 alpha, depth multiplier 參數
   - expansion 因子爲 6

##### 載入套件

In [1]:
from keras.models import Input, Model
from keras.layers import DepthwiseConv2D, Conv2D, BatchNormalization, ReLU, Add

Using TensorFlow backend.
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

##### 定義 Separable Convolution 函數 (請在此實做)

In [4]:
def InvertedRes(input, expansion):
    '''
    Args:
        input: input tensor
        expansion: expand filters size
    Output:
        output: output tensor
    '''
    # Depthwise Convolution
    # DepthwiseConv2D():深度可分離2D卷積
    """
    DepthwiseConv2D()參數說明:
    
    kernel_size=(3,3)，指定2D卷積窗口的高度和寬度。
    padding="same"
    """
    x = DepthwiseConv2D((3,3), padding='same')(input)
    x = BatchNormalization()(x) # BatchNormalization():批量標準化層
    x = ReLU()(x) #ReLU: 激活函數。
    
    # Pointwise Convolution
    # Conv2D():2D 卷積
    ''' Conv2D()參數說明'''
    """
    kernel=3 ，核張量。
    strides=(1,1) ，步長元組。
    """
    x = Conv2D(3, (1,1))(x)
    x = BatchNormalization()(x) # BatchNormalization():批量標準化層
    x = ReLU()(x) #ReLU: 激活函數。
    
    # Add a Layer
    # Add():計算輸入張量列表的和
    """
    Add():
    它接受一個張量的列表， 所有的張量必須有相同的輸入尺寸， 然後返回一個張量（和輸入張量尺寸相同）
    """
    x = Add()([input, x])
    
    return x


##### 建構模型

In [5]:
input = Input((64, 64, 3))
output = InvertedRes(input, 6)
model = Model(inputs=input, outputs=output)
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 64, 64, 3)    0                                            
__________________________________________________________________________________________________
depthwise_conv2d_2 (DepthwiseCo (None, 64, 64, 3)    30          input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_3 (BatchNor (None, 64, 64, 3)    12          depthwise_conv2d_2[0][0]         
__________________________________________________________________________________________________
re_lu_3 (ReLU)                  (None, 64, 64, 3)    0           batch_normalization_3[0][0]      
__________________________________________________________________________________________________
conv2d_2 (